In [6]:
import ortools
from ortools.constraint_solver import routing_enums_pb2
import ortools.constraint_solver.pywrapcp as pywrapcp
from PIL import Image
from IPython.display import display
import pandas as pd
from itertools import zip_longest
from geopy.distance import geodesic

In [7]:
teste = pd.read_excel(r"ara_adm (1).xlsx")

In [8]:
teste

,qtd,lat,long
0,0,-19.837913,-40.082170
1,5,-19.903273,-40.232890
2,2,-19.904354,-40.223015
3,2,-19.920553,-40.174908
4,2,-19.927172,-40.164630
...,...,...,...
138,5,-20.261312,-40.263690
139,3,-20.257410,-40.263832
140,2,-20.250269,-40.265118
141,1,-20.214258,-40.263565


In [9]:
teste['lat'] = teste['lat'].astype(str)
teste['long'] = teste['long'].astype(str)
teste['lat'] = teste['lat'].str.replace(",",".")
teste['long'] = teste['long'].str.replace(",",".")
teste['lat'] = teste['lat'].astype(float)
teste['long'] = teste['long'].astype(float)
teste['qtd'] = teste['qtd'].astype(int)

In [10]:
x = list(zip_longest(teste['lat'], teste['long'], fillvalue=None))

In [11]:
# Crie uma matriz de distância vazia
distance_matrix = []

# Calcule as distâncias entre todos os pares de coordenadas
for coord1 in x:
    distances = []
    for coord2 in x:
        distancia = geodesic(coord1, coord2).kilometers
        distances.append(distancia)
    distance_matrix.append(distances)

# Imprima a matriz de distância
# for i, row in enumerate(distance_matrix):
#     for j, distancia in enumerate(row):
#         print(f"Distância entre cidade {i} e cidade {j}: {distancia} km")


## Restrições de capacidade

restriçoes de demanda, quantidade de veículos, capacidade dos veiculos

## Rotas ciculares

In [13]:
"""Capacited Vehicles Routing Problem (CVRP)."""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model(distance_matrix):
    """Stores the data for the problem."""
    data = {}
    data["distance_matrix"] = distance_matrix
    data["demands"] = teste['qtd']
    data["vehicle_capacities"] = [46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46]
    data["num_vehicles"] = len(data["vehicle_capacities"])
    data["depot"] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data["demands"][node_index]
            plan_output += f" {node_index} Load({route_load}) -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        plan_output += f" {manager.IndexToNode(index)} Load({route_load})\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        plan_output += f"Load of the route: {route_load}\n"
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print(f"Total distance of all routes: {total_distance}m")
    print(f"Total load of all routes: {total_load}")

    
def verify_data(data):
    demanda = sum([x for x in data["demands"]])
    lugares = sum([x for x in data["vehicle_capacities"]])
    
    if lugares >= demanda:
        print(f"existem mais lugares que demanda")
        return True
        
    if demanda > lugares:
        print(f"Não existem lugares suficientes \n"
             f"{lugares} -- lugares, {demanda} --  demanda\n")
        return False
    

def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model(distance_matrix)

    x = verify_data(data)
    
    if x == False:
        print(x)
    
    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data["demands"][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data["vehicle_capacities"],  # vehicle maximum capacities
        True,  # start cumul to zero
        "Capacity",
    )

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters.time_limit.FromSeconds(1)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)


if __name__ == "__main__":
    main()

existem mais lugares que demanda
Objective: 996
Route for vehicle 0:
 0 Load(0) ->  0 Load(0)
Distance of the route: 0m
Load of the route: 0

Route for vehicle 1:
 0 Load(0) ->  133 Load(5) ->  104 Load(7) ->  90 Load(11) ->  102 Load(12) ->  111 Load(14) ->  101 Load(16) ->  100 Load(18) ->  99 Load(20) ->  89 Load(21) ->  98 Load(22) ->  97 Load(23) ->  96 Load(25) ->  110 Load(26) ->  109 Load(27) ->  95 Load(28) ->  108 Load(29) ->  107 Load(30) ->  81 Load(32) ->  82 Load(33) ->  83 Load(34) ->  84 Load(38) ->  85 Load(39) ->  86 Load(40) ->  88 Load(42) ->  87 Load(44) ->  125 Load(46) ->  0 Load(46)
Distance of the route: 132m
Load of the route: 46

Route for vehicle 2:
 0 Load(0) ->  119 Load(2) ->  121 Load(5) ->  114 Load(11) ->  113 Load(12) ->  141 Load(13) ->  112 Load(17) ->  140 Load(19) ->  139 Load(22) ->  138 Load(27) ->  137 Load(28) ->  136 Load(30) ->  135 Load(32) ->  134 Load(33) ->  103 Load(34) ->  58 Load(46) ->  0 Load(46)
Distance of the route: 140m
Load of 

In [14]:
x = list(zip_longest(teste['lat'], teste['long'], fillvalue=None))

In [15]:
# Crie uma matriz de distância vazia
distance_matrix = []

# Calcule as distâncias entre todos os pares de coordenadas
for coord1 in x:
    distances = []
    for coord2 in x:
        distancia = geodesic(coord1, coord2).kilometers
        distances.append(distancia)
    distance_matrix.append(distances)

# Imprima a matriz de distância
# for i, row in enumerate(distance_matrix):
#     for j, distancia in enumerate(row):
#         print(f"Distância entre cidade {i} e cidade {j}: {distancia} km")


## Rotas em linha

In [16]:
"""Simple Vehicles Routing Problem."""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model(distance_matrix):
    """Stores the data for the problem."""
    data = {}
    data["distance_matrix"] = distance_matrix
    data["num_vehicles"] = 9
    data["starts"] = [1, 2, 15, 16, 8, 10, 35, 56, 71]
    data["ends"] = [0, 0, 0, 0, 0, 0, 0, 0, 0]
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    max_route_distance = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += f" {manager.IndexToNode(index)} -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        plan_output += f"{manager.IndexToNode(index)}\n"
        plan_output += f"Distance of the route: {route_distance}m\n"
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print(f"Maximum of the route distances: {max_route_distance}m")


def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model(distance_matrix)

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["starts"], data["ends"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = "Distance"
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        2000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name,
    )
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)


if __name__ == "__main__":
    main()

Objective: 10780
Route for vehicle 0:
 1 -> 0
Distance of the route: 0m

Route for vehicle 1:
 2 -> 0
Distance of the route: 0m

Route for vehicle 2:
 15 ->  117 ->  116 ->  92 ->  91 ->  90 ->  100 ->  99 ->  97 ->  109 ->  95 ->  110 ->  101 ->  102 ->  103 ->  104 ->  136 ->  137 ->  138 ->  139 ->  140 ->  141 ->  124 ->  122 ->  121 ->  120 ->  119 ->  118 ->  132 ->  14 ->  20 ->  19 ->  13 ->  12 ->  21 ->  22 ->  23 -> 0
Distance of the route: 105m

Route for vehicle 3:
 16 -> 0
Distance of the route: 0m

Route for vehicle 4:
 8 ->  17 ->  11 ->  9 ->  18 ->  131 ->  130 ->  123 ->  114 ->  125 ->  126 ->  142 ->  113 ->  112 ->  89 ->  98 ->  96 ->  108 ->  107 ->  81 ->  82 ->  83 ->  84 ->  85 ->  86 ->  87 ->  88 ->  111 ->  135 ->  134 ->  133 ->  115 ->  105 ->  127 ->  93 ->  128 ->  106 ->  94 ->  129 ->  7 ->  5 ->  6 -> 0
Distance of the route: 104m

Route for vehicle 5:
 10 -> 0
Distance of the route: 0m

Route for vehicle 6:
 35 ->  36 ->  37 ->  25 ->  24 ->  75 ->

### Para 4 rotas

In [ ]:
# Abra a imagem
imagem = Image.open("Anotação 2023-11-06 141717.jpg")

# Exiba a imagem no notebook
display(imagem)

### Para 5 rotas

In [ ]:
# Abra a imagem
imagem = Image.open("Anotação 2023-11-06 141717 2.jpg")

# Exiba a imagem no notebook
display(imagem)